In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Python Standard Lib
import os.path

# PIP
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.applications.efficientnet import *

In [ ]:
# Image Directory in my Google Drive
image_dir = Path('/content/drive/MyDrive/MathML/A2_CNN/Images')

In [ ]:
# Load Dataset
# Folder: The FilePath name, Carrots, Rockets, will be labels for each image.

filepaths = list(image_dir.glob(r'**/*.jpg')) # jpg files in any folders
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

image_df = pd.concat([filepaths, labels], axis=1)

In [ ]:
image_df

,Filepath,Label
0,/content/drive/MyDrive/MathML/A2_CNN/Images/Ca...,Carrots
1,/content/drive/MyDrive/MathML/A2_CNN/Images/Ca...,Carrots
2,/content/drive/MyDrive/MathML/A2_CNN/Images/Ca...,Carrots
3,/content/drive/MyDrive/MathML/A2_CNN/Images/Ca...,Carrots
4,/content/drive/MyDrive/MathML/A2_CNN/Images/Ca...,Carrots
...,...,...
301,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
302,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
303,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
304,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets


In [ ]:
# Mix the dataset randomly
image_df = image_df.sample(frac=1).reset_index(drop = True)
image_df.head(8)

,Filepath,Label
0,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
1,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
2,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
3,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
4,/content/drive/MyDrive/MathML/A2_CNN/Images/Ca...,Carrots
5,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
6,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
7,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets


In [ ]:
# Image Pre-processing => Augmentation is required since too few images
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, # image -> 0~1 normalization
    horizontal_flip=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.1 # 10% of dataset is allocated to validation dataset
)

In [ ]:
# Image file path to image dataset
train_images = train_generator.flow_from_dataframe(
    dataframe=image_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical', # Classification
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=image_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

Found 276 validated image filenames belonging to 2 classes.
Found 30 validated image filenames belonging to 2 classes.


In [ ]:
# Classes: Carrots=0, Rockets=1
train_images.class_indices

{'Carrots': 0, 'Rockets': 1}

In [ ]:
# CNN MODEL
inputs = tf.keras.Input(shape=(224, 224, 3))

# 3 convolution layers
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D(2,2)(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(2,2)(x)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(2,2)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.2)(x) # 20% Dropout => Anti-overfitting

# 3 FC Layers
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)

#
outputs = tf.keras.layers.Dense(2, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Early-Stopping & Scaling
callbacks = [
    EarlyStopping(monitor='val_loss', mode='min', patience=10, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.1, patience=5, min_lr=0.000001, verbose=1)
]

In [ ]:
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=20,
    callbacks=callbacks
)

Epoch 1/50
9/9 [==============================] - 8s 606ms/step - loss: 0.7742 - accuracy: 0.4891 - val_loss: 0.6116 - val_accuracy: 0.7000 - lr: 0.0010
Epoch 2/50
9/9 [==============================] - 6s 645ms/step - loss: 0.5640 - accuracy: 0.7935 - val_loss: 0.4707 - val_accuracy: 0.7667 - lr: 0.0010
Epoch 3/50
9/9 [==============================] - 6s 661ms/step - loss: 0.3881 - accuracy: 0.8659 - val_loss: 0.6292 - val_accuracy: 0.7667 - lr: 0.0010
Epoch 4/50
9/9 [==============================] - 6s 640ms/step - loss: 0.3595 - accuracy: 0.8841 - val_loss: 0.4312 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 5/50
9/9 [==============================] - 6s 644ms/step - loss: 0.2867 - accuracy: 0.9094 - val_loss: 0.4566 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 6/50
9/9 [==============================] - 6s 631ms/step - loss: 0.2540 - accuracy: 0.9022 - val_loss: 0.4103 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 7/50
9/9 [==============================] - 6s 634ms/step - loss: 0.2455 - a

In [ ]:
model.save("/content/drive/MyDrive/MathML/A2_CNN/vanilla_cnn_model")